In [ ]:
###
### This notebook was run in a googlecolab environment using GPU
###

!pip install sentencepiece
!pip install transformers


import pandas as pd
import torch
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
import time

In [ ]:
device = torch.device( 'cuda')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
config = T5Config.from_pretrained('t5-base')

In [ ]:
def predictqueryfromdoc(doc):
    start = time.time()
    input_ids = tokenizer.encode(doc + "</s>", return_tensors='pt').to(device)
    
    if input_ids.size()[1]>512:
        input_ids = input_ids[:,:511]

    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_k=3,
        num_return_sequences=10)
    t = [tokenizer.decode(i, skip_special_tokens=True) for i in outputs]
    return t

In [ ]:
args = pd.read_csv("args.tsv", sep="\t")
args["pred"] = "none"
args.reset_index(drop=True, inplace=True)

In [ ]:
def predict(row):
    return predictqueryfromdoc(row["premises"])

args["pred"] = args.apply(predict, axis=1)

args.to_csv("args_docT5query.csv", sep=";", index=False)

In [ ]:
args["pred"] = [" ".join(queries).replace("'", "").replace('"', '') for queries in args["pred"]]
args["args_t5expansion"] =  args["premises"] + " " + args["pred"]
args.to_csv("t5_expansion.tsv", sep="\t", index=False)